<a href="https://colab.research.google.com/github/OlegLaiok/practise/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
from nltk.corpus import stopwords
import gensim.corpora as corpora
import pandas as pd
!pip install pymorphy2
import pymorphy2
import json
from gensim.models import LdaMulticore
from multiprocessing import Process, freeze_support
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models
import pickle 
import os
from nltk.util import ngrams
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from gensim.models.coherencemodel import CoherenceModel

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel("/content/drive/MyDrive/ColabNotebooks/Практика/workprogramsapp_expertisecomments_original.xlsx")
df

,id,comment_block,comment_text,comment_date,user_expertise_id
0,1.0,MA,Довольно мало информации связанной непосредств...,2021-02-19 16:39:00,2.0
1,2.0,PR,"Пожалуйста, укажите пререквизиты",2021-02-19 16:39:00,2.0
2,3.0,SE,Очень странно рассчитаны часы и всего два разд...,2021-02-19 16:40:00,2.0
3,4.0,TH,"Тем в разделах тоже очень мало, такое ощущение...",2021-02-19 16:41:00,2.0
4,5.0,SO,Где источники?,2021-02-19 16:41:00,2.0
...,...,...,...,...,...
2771,2827.0,EV,Здравствуйте! Я не вносила оценочное средство ...,2021-09-14 22:42:00,2173.0
2772,2828.0,MA,В описании факультатива с момента подачи заявк...,2021-09-14 22:47:00,2173.0
2773,2829.0,EV,Примеры тестов добавлены.,2021-09-15 00:14:00,2173.0
2774,2830.0,CO,"Анжелика, информацию внёс",2021-09-15 06:06:00,2226.0


In [ ]:
tags = {'MA':'Главное',
        'PR':'Пререквизиты',
        'SE':'Разделы',
        'TH':'Темы',
        'SO':'Источники',
        'EV':'Оценочные средства',
        'RE':'Результаты обучения',
        'CO':'Контрольные средства'
        }
comments_num = {'Главное': 0,
                'Пререквизиты': 0,
                'Разделы': 0,
                'Темы': 0,
                'Источники': 0,
                'Оценочные средства': 0,
                'Результаты обучения': 0,
                'Контрольные средства': 0
              }
for i in range(len(df['comment_block'])):
  df['comment_block'][i]=tags[df['comment_block'][i]]
  comments_num[df['comment_block'][i]] += 1
df

,id,comment_block,comment_text,comment_date,user_expertise_id
0,1.0,Главное,Довольно мало информации связанной непосредств...,2021-02-19 16:39:00,2.0
1,2.0,Пререквизиты,"Пожалуйста, укажите пререквизиты",2021-02-19 16:39:00,2.0
2,3.0,Разделы,Очень странно рассчитаны часы и всего два разд...,2021-02-19 16:40:00,2.0
3,4.0,Темы,"Тем в разделах тоже очень мало, такое ощущение...",2021-02-19 16:41:00,2.0
4,5.0,Источники,Где источники?,2021-02-19 16:41:00,2.0
...,...,...,...,...,...
2771,2827.0,Оценочные средства,Здравствуйте! Я не вносила оценочное средство ...,2021-09-14 22:42:00,2173.0
2772,2828.0,Главное,В описании факультатива с момента подачи заявк...,2021-09-14 22:47:00,2173.0
2773,2829.0,Оценочные средства,Примеры тестов добавлены.,2021-09-15 00:14:00,2173.0
2774,2830.0,Контрольные средства,"Анжелика, информацию внёс",2021-09-15 06:06:00,2226.0


In [ ]:
comments_num

{'Главное': 268,
 'Источники': 114,
 'Контрольные средства': 632,
 'Оценочные средства': 1226,
 'Пререквизиты': 157,
 'Разделы': 53,
 'Результаты обучения': 317,
 'Темы': 9}

Видим что больше всего правок в категории Оценочные средства, Контрольные средства и Результаты обучения, однако в других категориях так же много правок поэтому следует

In [ ]:
groups = df.groupby('comment_block')
tags_1 = ['Главное', 'Пререквизиты', 'Разделы', 'Темы', 'Источники', 'Оценочные средства', 'Результаты обучения', 'Контрольные средства']
d_frames = {}
for tag in tags_1:
  d_frames[tag] = groups.get_group(tag)
d_frames['Главное']


,id,comment_block,comment_text,comment_date,user_expertise_id
0,1.0,Главное,Довольно мало информации связанной непосредств...,2021-02-19 16:39:00,2.0
8,14.0,Главное,Отлично! Никаких нареканий нет!,2021-02-19 16:53:00,4.0
18,25.0,Главное,"Пожалуйста, добавьте описание по Вашей программе",2021-04-29 11:29:00,31.0
24,31.0,Главное,"Пожалуйста, добавьте описание",2021-04-29 14:41:00,25.0
30,41.0,Главное,"Пожалуйста, добавьте описание",2021-04-29 15:29:00,58.0
...,...,...,...,...,...
2728,2784.0,Главное,Так как вы поставили дополнительные 3 балла пе...,2021-09-13 21:04:00,1801.0
2730,2786.0,Главное,Замечание указано в пункте 6,2021-09-13 21:07:00,512.0
2735,2791.0,Главное,"Замечания указаны в пункте 6, 7 и 8",2021-09-13 21:23:00,2636.0
2757,2813.0,Главное,"обратите, пожалуйста, внимание, что общеунивер...",2021-09-14 18:59:00,2199.0


In [ ]:
comments=[]
for tag in tags_1:
  comments.append(d_frames[tag]['comment_text'].to_numpy())

In [ ]:
import nltk # уже знакомая нам библиотека nltk
from nltk.tokenize import word_tokenize # готовый токенизатор библиотеки nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import string
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def get_most_popular_words(comm):
  lowerCase = [sentence.lower() for sentence in comm] # приведение к нижнему регистру
  tokenization = [word_tokenize(sentence) for sentence in lowerCase] #разбиение предложений на токены
  Stopword = stopwords.words('russian') + ['пожалуйста', 'добрый', 'день', 'прошу', 'необходимо',"''","``"] #составление списка стоп слов
  Stopword.extend(string.punctuation) # добавление к стоп словам знаков пунктуации
  cleared_tokens = [[word for word in sentence if word not in Stopword] for sentence in normalized]
  dictionary = corpora.Dictionary(cleared_tokens)
  corpus = [dictionary.doc2bow(sent) for sent in cleared_tokens]
  vocab = list(dictionary.values()) #list of terms in the dictionary
  vocab_tf = [dict(i) for i in corpus]
  vocab_tf = list(pd.DataFrame(vocab_tf).sum(axis=0)) #list of term frequencies
  d = {'words':vocab, 'freq': vocab_tf}
  most_freq_word = pd.DataFrame.from_dict(d).sort_values('freq', ascending=False)
  return most_freq_word


In [ ]:
get_most_popular_words(comments[7]) #функция чтобы найти среди самых частых слов стоп слова

,words,freq
0,добавить,338.0
15,оценивание,295.0
47,критерий,288.0
69,билет,252.0
25,пример,208.0
...,...,...
397,объединить,1.0
398,подсказка,1.0
399,теоретический,1.0
401,07.06,1.0


In [ ]:
import warnings
warnings.filterwarnings('ignore')

def compute_coherence_values(corpus, texts, id2word):
    scores = {'Coherence Score':[],'k': range(2,30)}
    for k in range(2,30):
      lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=k) #применяем LDA модель
      coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
      coherence_lda = coherence_model_lda.get_coherence()
      scores['Coherence Score'].append(coherence_lda)
    optim_scores = pd.DataFrame.from_dict(scores).sort_values('Coherence Score', ascending=False)
    print(optim_scores)
    max_score = optim_scores.iloc[0,1]
    return max_score

def get_most_common_topics(comm):
  lowerCase = [sentence.lower() for sentence in comm]
  tokenization = [word_tokenize(sentence) for sentence in lowerCase]
  Stopword = stopwords.words('russian') + ['пожалуйста', 'добрый', 'день', 'здравствуйте', 'просить', 'необходимо',"''","``"]
  Stopword.extend(string.punctuation)  cleared_tokens = [[word for word in sentence if word not in Stopword] for sentence in tokenization]
  d = {}
  bigramss = []
  for i in cleared_tokens:
    if len(i)>2:
      a = list(nltk.ngrams(i,2))
      for n in a:
        bigramss.append(n)
    else:
      continue
  d['dictionary'] = cleared_tokens
  d['all bigrams'] = list(bigramss)
  bigram_fd = nltk.FreqDist(bigramss)
  d['most common bigrams'] = list(bigram_fd.most_common())
  trigramss = []
  for i in cleared_tokens:
    if len(i)>2:
      a = list(nltk.ngrams(i,3))
      for n in a:
        trigramss.append(n)
    else:
      continue
  d['all trigrams'] = list(trigramss)
  bigram_fd = nltk.FreqDist(trigramss)
  d['most common trigrams'] = list(bigram_fd.most_common())
  id2word = corpora.Dictionary(bigramss)
  d['id2word'] = id2word
  corpus = [id2word.doc2bow(text) for text in bigramss] #cоздаем корпус слов
  d['corpus'] = corpus
  n=compute_coherence_values(corpus, bigramss, id2word)
  d['optimum_topics_num'] = n
  lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=n) #рприменяем LDA модель
  d['lda_model'] = lda_model
  d['lda_topics']=lda_model.show_topics()
  return d


In [ ]:
'Главное', 'Пререквизиты', 'Разделы', 'Темы', 'Источники', 'Оценочные средства', 'Результаты обучения', 'Контрольные средства'
d = get_most_common_topics(comments[0]) #ищем основные правки в категории Главное
print('Number of comments ', comments_num[tags_1[0]] )
print(d['all bigrams'])
print(d['most common bigrams'])
print(d['all trigrams'])
print(d['most common trigrams'])
print( d['optimum_topics_num'])
print()
for i in d['lda_topics']:
  print(i)

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(d['lda_model'], d['corpus'], d['id2word'])
LDAvis_prepared

    Coherence Score   k
16         0.757242  18
9          0.751823  11
13         0.750731  15
15         0.750720  17
17         0.750623  19
10         0.749927  12
7          0.749429   9
24         0.748335  26
22         0.748265  24
19         0.748021  21
25         0.747978  27
21         0.747922  23
26         0.747245  28
12         0.747175  14
23         0.747096  25
4          0.746929   6
27         0.746261  29
18         0.745665  20
20         0.745092  22
14         0.743916  16
2          0.743541   4
5          0.743335   7
1          0.742901   3
8          0.742826  10
6          0.741885   8
11         0.740743  13
0          0.737582   2
3          0.737495   5
Number of comments  268
[('довольно', 'мало'), ('мало', 'информации'), ('информации', 'связанной'), ('связанной', 'непосредственно'), ('непосредственно', 'курсом'), ('курсом', 'советую'), ('советую', 'дополнить'), ('дополнить', 'описание'), ('отлично', 'никаких'), ('никаких', 'нареканий'), ('добавьте', 

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
7      0.142955  0.073809       1        1  8.700118
12     0.150709 -0.056038       2        1  7.551729
11    -0.165398  0.105029       3        1  7.381896
2     -0.146562  0.077543       4        1  7.050091
10    -0.037316 -0.234698       5        1  5.825810
15    -0.103168 -0.117988       6        1  5.823235
1     -0.112657  0.083813       7        1  5.637258
8      0.083046  0.097046       8        1  5.372816
14    -0.108527 -0.006982       9        1  5.285076
4      0.054043 -0.080665      10        1  5.198562
13    -0.174243  0.036761      11        1  5.085185
9      0.098327  0.046683      12        1  4.822891
6      0.090873  0.077144      13        1  4.790735
17     0.095098  0.004357      14        1  4.783112
0     -0.069572 -0.105079      15        1  4.509210
5      0.079270 -0.004131      16        1  4.290569
16     0.082756 -0.048249      17        1  4.098339
3      0.040367  0.051645      18        1  3.793368, topic_info=           Term        Freq       Total Category  logprob  loglift
8      описание  149.000000  149.000000  Default  30.0000  30.0000
22   английском   74.000000   74.000000  Default  29.0000  29.0000
23        языке   58.000000   58.000000  Default  28.0000  28.0000
15     добавить   40.000000   40.000000  Default  27.0000  27.0000
40        прошу   36.000000   36.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
121           —    0.699298    8.663571  Topic18  -4.6867   0.7551
139   заполнить    0.699298   15.621533  Topic18  -4.6867   0.1656
158           6    0.699298   32.580887  Topic18  -4.6867  -0.5695
163      пункте    0.699298   32.791038  Topic18  -4.6867  -0.5759
15     добавить    0.699298   40.421043  Topic18  -4.6867  -0.7851

[867 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
19        7  0.725139         1
19       16  0.241713         1
174      16  0.704428  14.03.01
190       1  0.313771         2
190       2  0.179298         2
...     ...       ...       ...
121      12  0.115426         —
121      13  0.115426         —
121      16  0.115426         —
121      17  0.115426         —
121      18  0.115426         —

[1012 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 13, 12, 3, 11, 16, 2, 9, 15, 5, 14, 10, 7, 18, 1, 6, 17, 4])

Самые частые правки: добавить описание дисциплины, добавить описание дисциплины на англ языке(если дисциплина реализуется на англ языке), добавить авторский состав

In [ ]:
d = get_most_common_topics(comments[1]) #ищем основные правки в категории Пререквизиты
print('Number of comments ', comments_num[tags_1[1]] )
print(d['all bigrams'])
print(d['most common bigrams'])
print(d['all trigrams'])
print(d['most common trigrams'])
print( d['optimum_topics_num'])
for i in d['lda_topics']:
  print(i)
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(d['lda_model'], d['corpus'], d['id2word'])
LDAvis_prepared

    Coherence Score   k
23         0.767902  25
22         0.767424  24
24         0.767078  26
25         0.765567  27
20         0.763689  22
26         0.763585  28
2          0.762415   4
21         0.761706  23
27         0.761028  29
18         0.758278  20
0          0.756223   2
4          0.756074   6
19         0.755198  21
6          0.754946   8
7          0.751580   9
17         0.750759  19
8          0.750392  10
12         0.750319  14
3          0.749784   5
16         0.749614  18
13         0.749523  15
11         0.749491  13
10         0.748674  12
9          0.747689  11
15         0.747330  17
14         0.747288  16
5          0.746027   7
1          0.743614   3
Number of comments  157
[('рекомендую', 'добавить'), ('добавить', 'пререквизиты'), ('рекомендую', 'добавить'), ('добавить', 'пререквизиты'), ('рекомендуется', 'добавить'), ('добавить', 'пререквизиты'), ('рекомендуется', 'добавить'), ('добавить', 'пререквизиты'), ('рекомендуется', 'дополнить'), ('дополни

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.060854  0.195842       1        1  11.696282
10     0.310179 -0.026900       2        1   7.186992
4     -0.182202  0.183561       3        1   6.861789
5      0.168178  0.177634       4        1   6.536585
17    -0.151880  0.182594       5        1   6.432544
24     0.053613  0.031874       6        1   5.286767
1      0.034783  0.079116       7        1   5.235772
16    -0.020660 -0.318806       8        1   5.235772
19     0.082847  0.088101       9        1   4.260163
3      0.111367 -0.005547      10        1   4.260163
0     -0.094555  0.049218      11        1   3.609962
21    -0.182943  0.040699      12        1   3.609756
11     0.120764 -0.165910      13        1   3.499318
22     0.087115 -0.025718      14        1   3.106947
12    -0.047639  0.023531      15        1   2.796542
23    -0.123558 -0.084660      16        1   2.308943
14     0.211392  0.057789      17        1   2.308943
15    -0.099790 -0.187209      18        1   2.308943
13     0.245231  0.021872      19        1   2.308943
9     -0.231206 -0.019637      20        1   1.983740
7      0.022487  0.074161      21        1   1.983740
6     -0.118029 -0.101683      22        1   1.983740
2     -0.218660 -0.045245      23        1   1.983740
20     0.003878 -0.164105      24        1   1.880582
18     0.080144 -0.060571      25        1   1.333333, topic_info=             Term       Freq      Total Category  logprob  loglift
2    пререквизиты  46.000000  46.000000  Default   30.000  30.0000
3   рекомендуется  21.000000  21.000000  Default   29.000  29.0000
0        добавить  57.000000  57.000000  Default   28.000  28.0000
11        учебный  25.000000  25.000000  Default   27.000  27.0000
10       сущности  17.000000  17.000000  Default   26.000  26.0000
..            ...        ...        ...      ...      ...      ...
17       измените   0.035498   8.503887  Topic25   -5.037  -1.1613
20       освоения   0.035498   5.538063  Topic25   -5.037  -0.7324
26       обновлен   0.035498   3.049395  Topic25   -5.037  -0.1357
27             уп   0.035498   3.471740  Topic25   -5.037  -0.2655
29    пререквизит   0.035498   2.915294  Topic25   -5.037  -0.0908

[1160 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
21        5  0.414903   анастасия
21       19  0.414903   анастасия
38        1  0.075945  английском
38        3  0.151889  английском
38        4  0.075945  английском
...     ...       ...         ...
39        6  0.227045       языке
39       11  0.113523       языке
39       13  0.113523       языке
39       17  0.113523       языке
39       21  0.113523       языке

[221 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 11, 5, 6, 18, 25, 2, 17, 20, 4, 1, 22, 12, 23, 13, 24, 15, 16, 14, 10, 8, 7, 3, 21, 19])

Добавить или дополнить пререквизиты(если дисциплина на англ то на англ языке), учебные сущности

In [ ]:
d1 = get_most_common_topics(comments[2]) #ищем основные правки в категории Разделы
print('Number of comments ', comments_num[tags_1[2]] )
print(d1['all bigrams'])
print(d1['most common bigrams'])
print(d1['all trigrams'])
print(d1['most common trigrams'])
print(d1['optimum_topics_num'])
for i in d1['lda_topics']:
  print(i)
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(d1['lda_model'], d1['corpus'], d1['id2word'], mds='mmds')
LDAvis_prepared

    Coherence Score   k
6          0.747742   8
14         0.747461  16
11         0.747133  13
15         0.743959  17
2          0.743085   4
24         0.742297  26
27         0.741502  29
13         0.741336  15
4          0.740739   6
8          0.740499  10
26         0.740105  28
18         0.739637  20
23         0.739453  25
5          0.739411   7
16         0.739166  18
19         0.738192  21
22         0.738009  24
17         0.736822  19
25         0.736453  27
3          0.736278   5
10         0.736245  12
9          0.736119  11
20         0.734480  22
21         0.733201  23
12         0.732149  14
7          0.727680   9
1          0.722288   3
0          0.703073   2
Number of comments  53
[('очень', 'странно'), ('странно', 'рассчитаны'), ('рассчитаны', 'часы'), ('часы', 'раздела'), ('раздела', 'никуда'), ('никуда', 'годится'), ('соответствии', 'уп'), ('уп', 'предусмотрено'), ('предусмотрено', '16'), ('16', 'часов'), ('часов', 'лекционных'), ('лекционных', 'занятий'

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.137692  0.149330       1        1  16.834431
5      0.050616  0.240230       2        1  15.527417
2      0.171741 -0.076351       3        1  13.042593
6     -0.054908 -0.145900       4        1  12.650174
0      0.177487  0.081890       5        1  11.455884
4     -0.033886  0.003134       6        1  11.037723
3     -0.245152 -0.077082       7        1  10.498347
1      0.071793 -0.175253       8        1   8.953432, topic_info=            Term       Freq      Total Category  logprob  loglift
10            16  24.000000  24.000000  Default  30.0000  30.0000
70         плану  20.000000  20.000000  Default  29.0000  29.0000
35  практических  21.000000  21.000000  Default  28.0000  28.0000
13       занятий  19.000000  19.000000  Default  27.0000  27.0000
69      учебному  17.000000  17.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
36            24   1.646889  17.972222   Topic8  -3.6946   0.0232
13       занятий   1.646564  19.279338   Topic8  -3.6948  -0.0472
11         часов   0.873088  21.657619   Topic8  -4.3292  -0.7979
8             уп   0.872393  15.497766   Topic8  -4.3300  -0.4641
7   соответствии   0.871545   6.641993   Topic8  -4.3310   0.3822

[372 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
161       2  0.891699  01.04.02
45        1  0.383903         1
45        6  0.383903         1
116       4  0.450003        10
80        2  0.447912       108
...     ...       ...       ...
121       4  0.657565       это
124       5  0.396933     языка
124       7  0.396933     языка
132       2  0.423408     языку
132       7  0.423408     языку

[380 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 6, 3, 7, 1, 5, 4, 2])

Уточнить количество выделенных часов(не свопадает с УП)

In [ ]:
d = get_most_common_topics(comments[3]) #ищем основные правки в категории Темы
print('Number of comments ', comments_num[tags_1[3]] )
print(d['all bigrams'])
print(d['most common bigrams'])
print(d['all trigrams'])
print(d['most common trigrams'])
print( d['optimum_topics_num'])
for i in d['lda_topics']:
  print(i)
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(d['lda_model'], d['corpus'], d['id2word'], mds='mmds')
LDAvis_prepared

    Coherence Score   k
15         0.785495  17
13         0.781595  15
14         0.779776  16
11         0.779488  13
12         0.776741  14
8          0.776622  10
9          0.776218  11
10         0.775339  12
7          0.772433   9
16         0.771680  18
3          0.769186   5
19         0.765833  21
4          0.765275   6
21         0.764902  23
23         0.764405  25
22         0.763779  24
6          0.763539   8
5          0.763042   7
24         0.762222  26
26         0.761946  28
20         0.761813  22
25         0.761700  27
18         0.761621  20
2          0.760202   4
27         0.760066  29
17         0.759419  19
1          0.755890   3
0          0.752220   2
Number of comments  9
[('разделах', 'очень'), ('очень', 'мало'), ('мало', 'такое'), ('такое', 'ощущение'), ('ощущение', 'это'), ('это', 'какая-то'), ('какая-то', 'тестовая'), ('тестовая', 'рпд'), ('неплохо', 'правильно'), ('правильно', 'подобраны'), ('подобраны', 'онлайн'), ('онлайн', 'курсы'), ('уточни

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.025911  0.291961       1        1  13.071895
4     -0.255195 -0.002262       2        1   8.978328
3      0.095277  0.218441       3        1   7.808738
15    -0.221219  0.114966       4        1   7.808737
13    -0.125616  0.212939       5        1   6.639147
11    -0.050182 -0.256149       6        1   6.639147
12     0.103462 -0.215341       7        1   6.639147
8      0.184682 -0.056553       8        1   5.469556
5      0.233023  0.091479       9        1   5.469556
14     0.197962 -0.106263      10        1   5.469556
10     0.090668  0.057165      11        1   4.884761
16    -0.159694 -0.062740      12        1   4.299966
7     -0.113879 -0.141569      13        1   4.299966
9     -0.021980  0.061225      14        1   4.299966
1      0.051848 -0.166716      15        1   3.130375
0     -0.042935 -0.035996      16        1   3.130375
2      0.007870 -0.004587      17        1   1.960784, topic_info=          Term      Freq     Total Category  logprob  loglift
22       часов  6.000000  6.000000  Default  30.0000  30.0000
15    уточните  3.000000  3.000000  Default  29.0000  29.0000
21  аудиторных  3.000000  3.000000  Default  28.0000  28.0000
20     хватает  3.000000  3.000000  Default  27.0000  27.0000
3        такое  2.000000  2.000000  Default  26.0000  26.0000
..         ...       ...       ...      ...      ...      ...
7     тестовая  0.042986  2.269557  Topic17  -3.9512  -0.0346
14     разделе  0.042986  2.076589  Topic17  -3.9512   0.0542
24      раздел  0.042986  2.276553  Topic17  -3.9512  -0.0377
27    изучения  0.042986  2.267666  Topic17  -3.9512  -0.0338
30  дисциплина  0.042986  2.180218  Topic17  -3.9512   0.0055

[804 rows x 6 columns], token_table=      Topic      Freq Term
term                      
33        4  0.508869  144
33        5  0.508869  144
16        3  0.522458    2
16       10  0.522458    2
32        4  0.450236   48
...     ...       ...  ...
29        1  0.498104  эта
29        3  0.498104  эта
5         2  0.293988  это
5         4  0.293988  это
5        11  0.293988  это

[96 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 4, 16, 14, 12, 13, 9, 6, 15, 11, 17, 8, 10, 2, 1, 3])

Слишком малое кол-во данных, пока можно предложить: мало тем, праильно ли подобраны онлайн курсы, хватит ли выделенных часов на раздел

In [ ]:
d = get_most_common_topics(comments[4]) #ищем основные правки в категории Источники
print('Number of comments ', comments_num[tags_1[4]] )
print(d['all bigrams'])
print(d['most common bigrams'])
print(d['all trigrams'])
print(d['most common trigrams'])
print( d['optimum_topics_num'])
for i in d['lda_topics']:
  print(i)
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(d['lda_model'], d['corpus'], d['id2word'])
LDAvis_prepared

    Coherence Score   k
3          0.768962   5
2          0.763801   4
1          0.762886   3
4          0.758973   6
5          0.756863   7
7          0.755779   9
8          0.755372  10
9          0.752677  11
6          0.752128   8
0          0.750192   2
10         0.748740  12
12         0.746275  14
11         0.744420  13
13         0.743849  15
14         0.742391  16
18         0.741055  20
20         0.739577  22
21         0.739033  23
24         0.738593  26
23         0.737876  25
27         0.736362  29
22         0.736268  24
15         0.735931  17
17         0.735131  19
16         0.735094  18
26         0.734574  28
25         0.733505  27
19         0.731237  21
Number of comments  114
[('указать', 'ссылку'), ('ссылку', 'конкретную'), ('конкретную', 'книгу'), ('укажите', 'ссылку'), ('ссылку', 'эбс'), ('дополнить', 'ссылки'), ('ссылки', 'эбс'), ('дополнить', 'ссылками'), ('ссылками', 'эбс'), ('добавьте', 'источник'), ('источник', 'ссылку'), ('ссылку', 'эбс'), ('

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.198517  0.002082       1        1  27.890738
0      0.053851 -0.184625       2        1  25.722369
3      0.040371  0.003529       3        1  17.176330
1      0.102465  0.098230       4        1  15.488337
4      0.001831  0.080784       5        1  13.722226, topic_info=           Term       Freq      Total Category  logprob  loglift
5           эбс  46.000000  46.000000  Default  30.0000  30.0000
0        ссылку  78.000000  78.000000  Default  29.0000  29.0000
11     добавить  37.000000  37.000000  Default  28.0000  28.0000
9      добавьте  24.000000  24.000000  Default  27.0000  27.0000
29   рекомендую  13.000000  13.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
15        поиск   1.062561   2.681463   Topic5  -4.4363   1.0605
30        дубль   1.062557   2.681262   Topic5  -4.4363   1.0605
25       первый   1.086441   8.572665   Topic5  -4.4140  -0.0795
27        нужно   1.069182   4.017318   Topic5  -4.4301   0.6624
137      список   1.062311   2.681735   Topic5  -4.4365   1.0601

[241 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
119       2  0.391778   2018
119       4  0.391778   2018
49        1  0.409449      5
49        3  0.409449      5
47        4  0.558654   6-ти
...     ...       ...    ...
66        1  0.229632  языке
66        5  0.688895  языке
115       2  0.234569      –
115       3  0.234569      –
115       4  0.469139      –

[244 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2, 5])

Добавить источники(на англ), ссылки на эбс

In [ ]:
d = get_most_common_topics(comments[5]) #ищем основные правки в категории Оценочные средства
print('Number of comments ', comments_num[tags_1[5]] )
print(d['all bigrams'])
print(d['most common bigrams'])
print(d['all trigrams'])
print(d['most common trigrams'])
print( d['optimum_topics_num'])
for i in d['lda_topics']:
  print(i)
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(d['lda_model'], d['corpus'], d['id2word'])
LDAvis_prepared

    Coherence Score   k
1          0.722160   3
4          0.720251   6
3          0.718736   5
0          0.716830   2
2          0.715083   4
6          0.707914   8
9          0.704925  11
7          0.701497   9
16         0.700194  18
25         0.699740  27
24         0.698960  26
21         0.698702  23
11         0.696693  13
27         0.696242  29
8          0.696133  10
22         0.695899  24
15         0.695252  17
20         0.695044  22
5          0.695003   7
23         0.694782  25
14         0.694290  16
13         0.693680  15
17         0.691784  19
10         0.691538  12
18         0.691379  20
26         0.691349  28
12         0.688759  14
19         0.683421  21
Number of comments  1226
[('те', 'замечания'), ('замечания', 'предыдущим'), ('предыдущим', 'пунктам'), ('странное', 'название'), ('название', 'оценочных'), ('оценочных', 'средств'), ('средств', 'ощущение'), ('ощущение', 'назначены'), ('назначены', 'уникальные'), ('уникальные', 'названия'), ('добавьте', 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.150951  0.146772       1        1  36.320766
1     -0.076705 -0.182575       2        1  33.696553
2      0.227655  0.035804       3        1  29.982681, topic_info=             Term        Freq       Total Category  logprob  loglift
179         прошу  559.000000  559.000000  Default  30.0000  30.0000
197    оценивания  671.000000  671.000000  Default  29.0000  29.0000
362         шкалу  814.000000  814.000000  Default  28.0000  28.0000
24         пример  426.000000  426.000000  Default  27.0000  27.0000
385    технологию  315.000000  315.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
556     оценочным   46.316780   77.461815   Topic3  -5.0571   0.6903
149       задание   46.882984   84.255921   Topic3  -5.0450   0.6183
70   лабораторной   51.489911  170.656995   Topic3  -4.9512   0.0063
552      внимание   45.931438   98.948622   Topic3  -5.0655   0.4371
100      описании   46.400823  210.853399   Topic3  -5.0553  -0.3093

[191 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
205       3  0.950728         ...
257       1  0.969990           1
257       2  0.022823           1
257       3  0.005706           1
1687      1  0.041261  10.09.2021
...     ...       ...         ...
495       2  0.979028           “
495       3  0.010198           “
497       1  0.010231           ”
497       2  0.982189           ”
497       3  0.010231           ”

[395 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

Добавить шкалу и критерии оценивания, шаблон проведения лабораторной работы, пример отчета(задания) по лабораторной работе, форму(технологию) проведения работы

In [ ]:
d = get_most_common_topics(comments[6]) #ищем основные правки в категории Результаты обучения
print('Number of comments ', comments_num[tags_1[6]] )
print(d['all bigrams'])
print(d['most common bigrams'])
print(d['all trigrams'])
print(d['most common trigrams'])
print( d['optimum_topics_num'])
for i in d['lda_topics']:
  print(i)
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(d['lda_model'], d['corpus'], d['id2word'])
LDAvis_prepared

    Coherence Score   k
23         0.735599  25
20         0.731146  22
13         0.730411  15
22         0.729429  24
25         0.728939  27
2          0.727731   4
21         0.727658  23
19         0.727105  21
27         0.726994  29
11         0.726860  13
26         0.726807  28
15         0.726740  17
12         0.725938  14
17         0.725218  19
6          0.724461   8
5          0.724257   7
18         0.724102  20
10         0.724079  12
24         0.723651  26
8          0.722394  10
16         0.721317  18
3          0.720634   5
14         0.719509  16
9          0.717936  11
7          0.717069   9
4          0.709540   6
1          0.708404   3
0          0.690981   2
Number of comments  317
[('каждого', 'результата'), ('результата', 'должно'), ('должно', 'указано'), ('указано', 'оценочное'), ('оценочное', 'средство'), ('каждого', 'результата'), ('результата', 'должно'), ('должно', 'указано'), ('указано', 'оценочное'), ('оценочное', 'средство'), ('каждой', 'учебной')

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
18    -0.007592  0.202968       1        1  8.403963
7      0.255428  0.057448       2        1  7.442506
23    -0.141975  0.125435       3        1  6.883395
19     0.014753 -0.146379       4        1  6.004644
17     0.053550 -0.135813       5        1  5.741538
20    -0.110191  0.219093       6        1  4.676080
1     -0.185462 -0.146522       7        1  4.382354
15     0.135605 -0.130465       8        1  4.361860
6      0.052130  0.046293       9        1  4.200175
14     0.052420  0.196030      10        1  3.820653
8     -0.058202 -0.082651      11        1  3.714571
0      0.276463  0.027601      12        1  3.688969
24    -0.197533 -0.135023      13        1  3.648303
10    -0.069680 -0.068086      14        1  3.506958
12     0.055388  0.086406      15        1  3.403724
16    -0.064580  0.057110      16        1  3.399027
11     0.032513 -0.131690      17        1  2.957556
4     -0.024338 -0.074404      18        1  2.949503
5     -0.049322  0.035542      19        1  2.803124
9     -0.037664  0.151398      20        1  2.525205
13    -0.124445 -0.123133      21        1  2.405077
21    -0.151597  0.061457      22        1  2.383895
2      0.162368 -0.015342      23        1  2.349817
22     0.257637 -0.083944      24        1  2.189790
3     -0.125674  0.006671      25        1  2.157313, topic_info=         Term        Freq       Total Category  logprob  loglift
11   средства  352.000000  352.000000  Default  30.0000  30.0000
56     каждый  195.000000  195.000000  Default  29.0000  29.0000
28      какие  240.000000  240.000000  Default  28.0000  28.0000
55  развивают  142.000000  142.000000  Default  27.0000  27.0000
53  заполните  104.000000  104.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
84     должны    0.868494   29.114512  Topic25  -4.4397   0.3241
28      какие    5.459940  240.324260  Topic25  -2.6013   0.0518
10  оценочные    7.987651  416.497362  Topic25  -2.2209  -0.1177
11   средства    3.373766  352.225378  Topic25  -3.0827  -0.8119
55  развивают    1.703585  142.039926  Topic25  -3.7660  -0.5871

[969 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
125       5  0.354620         1
125      10  0.354620         1
100      19  0.369471        1.
100      20  0.369471        1.
227      21  0.550612  10.09.21
...     ...       ...       ...
204      16  0.217410       это
204      17  0.217410       это
204      19  0.072470       это
204      22  0.072470       это
204      24  0.072470       это

[935 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 8, 24, 20, 18, 21, 2, 16, 7, 15, 9, 1, 25, 11, 13, 17, 12, 5, 6, 10, 14, 22, 3, 23, 4])

Добавить какие оценочные средства равзвивают каждый пререквизит(постреквизит), добавить оценочные средства к каждой сущности

In [ ]:
d = get_most_common_topics(comments[7]) #ищем основные правки в категории Контрольные средства
print('Number of comments ', comments_num[tags_1[7]] )
print(d['all bigrams'])
print(d['most common bigrams'])
print(d['all trigrams'])
print(d['most common trigrams'])
print( d['optimum_topics_num'])
for i in d['lda_topics']:
  print(i)
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(d['lda_model'], d['corpus'], d['id2word'])
LDAvis_prepared

    Coherence Score   k
2          0.724505   4
6          0.721944   8
17         0.721865  19
13         0.720085  15
15         0.718978  17
12         0.717264  14
9          0.716572  11
25         0.716406  27
21         0.716340  23
23         0.715964  25
22         0.715943  24
26         0.715822  28
11         0.714955  13
27         0.713857  29
18         0.713814  20
20         0.713572  22
16         0.712425  18
5          0.711838   7
0          0.711236   2
3          0.710708   5
24         0.710627  26
19         0.710335  21
7          0.707132   9
14         0.706928  16
4          0.702226   6
10         0.702031  12
8          0.701198  10
1          0.698938   3
Number of comments  632
[('очень', 'пусто'), ('пусто', 'постарайтесь'), ('постарайтесь', 'что-нибудь'), ('что-нибудь', 'добавить'), ('добавьте', 'название'), ('название', 'наименования'), ('наименования', 'экзамена'), ('одна', 'шкала'), ('шкала', 'оценивания'), ('оценивания', 'предусматривает'), ('преду

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.139077  0.138567       1        1  27.585648
0      0.113090  0.072894       2        1  25.361271
3     -0.131433 -0.153420       3        1  24.138189
1      0.157420 -0.058040       4        1  22.914891, topic_info=         Term        Freq       Total Category  logprob  loglift
33     пример  438.000000  438.000000  Default  30.0000  30.0000
22      шкалу  375.000000  375.000000  Default  29.0000  29.0000
5    добавьте  269.000000  269.000000  Default  28.0000  28.0000
82      также  185.000000  185.000000  Default  27.0000  27.0000
4    добавить  234.000000  234.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
54   описание   58.794496  259.535782   Topic4  -3.8686  -0.0115
406        20   26.704350   47.899382   Topic4  -4.6578   0.8891
34        т.к   34.763807   86.935200   Topic4  -4.3941   0.5568
81     билета   41.978338  380.549087   Topic4  -4.2055  -0.7311
22      шкалу   41.351665  375.613518   Topic4  -4.2206  -0.7331

[244 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
18        1  0.008224         0
18        2  0.904673         0
18        3  0.016449         0
18        4  0.074019         0
851       3  0.053278  09.07.21
...     ...       ...       ...
216       4  0.661898         “
218       1  0.020535         ”
218       2  0.390161         ”
218       3  0.574975         ”
218       4  0.020535         ”

[488 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2])

Добавить список вопросов к зачету/экзамену, шкалу(критерии) оценивания